# Причиность

In [ ]:
import warnings
warnings.filterwarnings("ignore")

try:
    import google.colab
    IN_COLAB = True
except:
    IN_COLAB = False

if IN_COLAB:
    !wget -q -O requirements.txt https://raw.githubusercontent.com/Intelligent-Systems-Phystech/psad/master/seminars/sem11/requirements.txt
    !pip install -qU -r requirements.txt
    !pip install -qU matplotlib==2.2.3

## Библиотеки

In [ ]:
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt

from statsmodels.regression.linear_model import OLS
import statsmodels.api as sm

import pandas as pd
import seaborn as sns

from causality.estimation.adjustments import AdjustForDirectCauses
from causality.estimation.nonparametric import CausalEffect
from causality.inference.search import IC
from causality.inference.independence_tests import MutualInformationTest
from causalgraphicalmodels import CausalGraphicalModel

import dowhy
from dowhy import CausalModel

## Warmup

### Связь в линейной регрессии

#### Fork

Смоделируем зависимости,соответствующие типу Fork.

Заметим, что здесь есть только одна переменная, которая является экзогенной (Z), то есть она не зависит ни от какой другой переменной в системе (не зависит от X, Y, эндогенных переменных).

$$Z->X, Z->Y, X->Y$$

![title](https://github.com/Intelligent-Systems-Phystech/psad/blob/master/seminars/sem11/data/fork.png?raw=1)

In [ ]:
rs = np.random.RandomState(42)
N = 10000
Z = rs.randn(10000)
X = 0.5 * Z + rs.randn(10000)
Y = 0.3 * Z + 0.4 * X + rs.randn(10000)

Построим разные регрессии регрессии Y ~ X, Y ~ Z + X, Y ~ Z. Что можно сказать о 95% доверительном интервале (confint) на коэффициент регрессии перед X в двух этих моделях?

In [ ]:
l1 = OLS(Y, np.vstack([X]).T).fit()
l1.summary()

In [ ]:
l1 = OLS(Y, np.vstack([X, Z]).T).fit()
l1.summary()

In [ ]:
l1 = OLS(Y, np.vstack([Z]).T).fit()
l1.summary()

### Collider

Рассмотрим collider:
```
X -> Z
X -> Y
Y -> Z
```

![title](https://github.com/Intelligent-Systems-Phystech/psad/blob/master/seminars/sem11/data/collider.png?raw=1)

In [ ]:
X = rs.randn(N)
Y = 0.7 * X + rs.randn(N)
Z = 1.2 * X + 0.6 * Y + rs.randn(N)

In [ ]:
l1 = OLS(Y, np.vstack([X]).T).fit()
l1.summary()

In [ ]:
l1 = OLS(Y, np.vstack([X, Z]).T).fit()
l1.summary()

In [ ]:
l1 = OLS(Y, np.vstack([Z]).T).fit()
l1.summary()

### DAGs (Directed Acyclic Graphs)

#### Рассмотрим простой граф

In [ ]:
G = CausalGraphicalModel(nodes=['U', 'Z', 'X', 'Y'], edges=[
    ('U', 'Y'),
    ('U', 'X'),
    ('X', 'Z'),
    ('Z', 'Y')
])
G.draw()

#### Найдем все пути в графе (без циклов)

In [ ]:
list(nx.all_simple_paths(G.dag, 'X', 'Y'))

#### Все пути, если считать граф не ориентированным

In [ ]:
list(nx.all_simple_paths(G.dag.to_undirected(), 'X', 'Y'))

#### Рассмотрим все зависимости

In [ ]:
G.get_all_independence_relationships()

#### Сгенерим распределение

In [ ]:
G.get_distribution()

#### Обусловимся на переменную X

In [ ]:
G.do('X').draw()

In [ ]:
G.draw()

In [ ]:
G.get_all_backdoor_adjustment_sets('Y','X')

In [ ]:
G.get_all_frontdoor_adjustment_sets('Y','X')

In [ ]:
G.get_all_frontdoor_adjustment_sets('X','Y')

## Inductive search

Допустим, у нас есть несколько событий:

1. Продажа мороженного (ICE)
2. Количество преступлений (CRIMES)
3. Количество полицейских на тысячу человек (POLICE)
4. Средняя температура (TEMP)
5. Количество скачиваний браузера IE (IE)
6. Количество зараженных компьютеров (COMP)
7. Уровень загрязнения воздуха (AIR)

P.S. Полная синтетика!

In [ ]:
G = CausalGraphicalModel(nodes=['ICE','CRIMES','POLICE', 'TEMP','IE','COMP', 'AIR'], 
edges=[('TEMP','ICE'),  ('TEMP','CRIMES'), ('IE','COMP'), ('POLICE', 'CRIMES')])
G.draw()

In [ ]:
rs = np.random.RandomState(42)
police = rs.uniform(low=1, high=100, size=1000)
temp = rs.uniform(low=-40, high=40, size=1000)
air = rs.uniform(size=1000)
crimes = (temp + 40) / police
ice = temp + 40
ie = rs.uniform(size=1000)
comp = ie**2
dataframe = pd.DataFrame({'police':police, 'temp':temp, 'air':air, 'crimes':crimes, 'ice':ice, 'ie':ie, 'comp':comp})

In [ ]:
dataframe

In [ ]:
class OLS_test():
    def __init__(self, y, x, z, data, alpha):
        self.regression = sm.OLS.from_formula('{0}~{1}'.format(y[0], '+'.join(x + z)), data)
        self.result = self.regression.fit()
        self.x = x
        self.y = y
        self.z = z
        self.alpha = alpha
        print(y, x, z)

    def independent(self):
        to_fisher = '(' + ' ,'.join([x_ + '=0' for x_ in self.x]) + ')'
        return self.result.f_test(to_fisher).pvalue > self.alpha

In [ ]:
variable_types = {'police' : 'd', 
                  'temp' : 'd',
                  'air' : 'c',
                  'ice' : 'd',
                  'crimes' : 'd',
                  'ie':'c',
                  'comp':'c'}

ic_algorithm = IC(OLS_test, alpha=0.05)
graph = ic_algorithm.search(dataframe, variable_types)

In [ ]:
nx.draw_networkx(graph)

In [ ]:
G = CausalGraphicalModel(nodes=['ICE','CRIMES','POLICE', 'TEMP','IE','COMP', 'AIR'], 
edges=[('TEMP','ICE'),  ('TEMP','CRIMES'), ('IE','COMP'), ('POLICE', 'CRIMES')])
G.draw()

In [ ]:
graph.edges

## Задача про курение

In [ ]:
smoke = []
cancer = []
tar = []

smoke += [1] * 323
cancer += [0]*323
tar+=[1]*323

smoke+=[0]
cancer+=[0]
tar+=[1]

smoke+=[1]*57
cancer+=[1]*57
tar+=[1]*57

smoke+=[0]*19
cancer+=[1]*19
tar+=[1]*19


smoke+=[1]*18
cancer+=[0]*18
tar+=[0]*18

smoke+=[0]*38
cancer+=[0]*38
tar+=[0]*38

smoke+=[1]*2
cancer+=[1]*2
tar+=[0]*2

smoke+=[0]*342
cancer+=[1]*342
tar+=[0]*342

dataframe = pd.DataFrame({'smoke':smoke, 'tar':tar, 'cancer':cancer})
dataframe.sample(5)

### Строим граф

In [ ]:
g = nx.DiGraph()

In [ ]:
g.add_nodes_from(['smoke','cancer','tar'])
g.add_edges_from([('tar','cancer'),('smoke','tar')])

In [ ]:
nx.draw_networkx(g)

### Иследуем причинность

In [ ]:
adjustment = AdjustForDirectCauses()
admissable_set = adjustment.admissable_set(g,['tar'], ['cancer'])
admissable_set

In [ ]:
dataframe.columns

In [ ]:
effect = CausalEffect(dataframe.sample(299), ['smoke'], ['cancer'], 
                      variable_types={'smoke': 'u', 'cancer': 'u', 'tar': 'u'},
                      admissable_set=[])

In [ ]:
smoke = pd.DataFrame({'smoke': [1], 'cancer': [1]})
nsmoke = pd.DataFrame({'smoke': [0], 'cancer': [1]})

In [ ]:
effect.pdf(smoke) - effect.pdf(nsmoke)

## DoWhy

### Вилка

In [ ]:
rs = np.random.RandomState(42)
X = [] 
Y = []
Z = []

Z+=[1]*100
Z+=[0]*100
X = [z if rs.uniform()<0.85 else rs.randint(low=0, high=2) for z in Z]
Y = [z if rs.uniform()<0.85 else rs.randint(low=0, high=2) for z in Z]

dataframe = pd.DataFrame({'X':X, 'Y':Y, 'Z':Z})
dataframe.sample(5)

In [ ]:
model_xy = CausalModel(
    data=dataframe,
    treatment=['X'],
    outcome=["Y"])
model_xy.interpret()

In [ ]:
model_xyz = CausalModel(
    data=dataframe,
    treatment=['X'],
    outcome=["Y"], common_causes=['Z']  )
model_xyz.interpret()

#### Определяем эффект

In [ ]:
print(model_xy.identify_effect())

In [ ]:
print(model_xyz.identify_effect())

#### Размер эффекта

In [ ]:
print(
    model_xy.estimate_effect(
        model_xy.identify_effect(), 
        method_name='backdoor.linear_regression', 
        test_significance=True))

In [ ]:
print(
    model_xyz.estimate_effect(
        model_xyz.identify_effect(), 
        method_name='backdoor.linear_regression', 
        test_significance=True, ))

### Усложняем модель
- Z->X->Y
- U->X, U->Y,

где U --- это скрытая переменная, при оценке эффекта мы ее не видим, но предполагаем, что она существует

In [ ]:
rs = np.random.RandomState(42)
X = [] 
Y = []
Z = []
U = []

U+=[1]*100
U+=[0]*100
Z += [1]*100
Z += [0] * 100
rs.shuffle(Z) # Z - еще одна переменная, которая влияет на X. Чтобы она не совпадала с U - помешаем.

X = [int((z+u)/2) if rs.uniform()<0.85 else rs.randint(low=0, high=2) for u,z in zip(U, Z)]
Y = [int((x+u)/2) for x,u in zip(X,U)]

dataframe = pd.DataFrame({'X':X, 'Y':Y, 'Z':Z})
dataframe.sample(5)

In [ ]:
model_xy = CausalModel(
    data=dataframe,
    treatment=['X'],
    outcome=["Y"]   )
model_xy.interpret()

In [ ]:
print(model_xy.identify_effect())

In [ ]:
print(
    model_xy.estimate_effect(
        model_xy.identify_effect(), 
        method_name='backdoor.linear_regression', 
        test_significance=True))

In [ ]:
causal_graph = """digraph {
X[label="X"];
Y[label="Y"];
Z[label="Z"];
U[label="Unobserved Confounders",observed="no"];
U->{X;Y};
X->{Y};
Z->{X};
}"""

model = CausalModel(
    data=dataframe,
    treatment=['X'],
    instruments=['Z'],
    outcome=["Y"],
    graph=causal_graph)
model.interpret()

In [ ]:
print(model.identify_effect())

In [ ]:
print(
    model.estimate_effect(
        model.identify_effect(), 
        method_name='iv.instrumental_variable', 
        test_significance=True))

Теперь немного поменяем данные:
- Пусть Y в реальности зависит от скрытой переменной U но не зависит от X
- Корреляция X и Y при этом объясняется именно общим влиянием U

In [ ]:
rs = np.random.RandomState(42)
X = [] 
Y = []
Z = []
U = []

U+=[1]*100
U+=[0]*100
Z += [1]*100
Z += [0] * 100
rs.shuffle(Z)

X = [int((z+u)/2) if rs.uniform()<0.85 else rs.randint(low=0, high=2) for u,z in zip(U, Z)]
Y = [u for x,u in zip(X,U)]

dataframe = pd.DataFrame({'X':X, 'Y':Y, 'Z':Z})
dataframe.sample(5)

In [ ]:
model = CausalModel(
    data=dataframe,
    treatment=['X'],
    outcome=["Y"])
model.interpret()

In [ ]:
print(model.identify_effect())

In [ ]:
print(
    model.estimate_effect(
        model.identify_effect(), 
        method_name='backdoor.linear_regression', 
        test_significance=True))

In [ ]:
model = CausalModel(
    data=dataframe,
    treatment=['X'],
    instruments=['Z'],
    outcome=["Y"],
    graph=causal_graph)
model.interpret()

In [ ]:
print(model.identify_effect())

In [ ]:
print(
    model.estimate_effect(
        model.identify_effect(), 
        method_name='iv.instrumental_variable', 
        test_significance=True))

## Социо-Экономический Статус

In [ ]:
causal_graph = """digraph {
X[label="X"];
Y[label="Y"];
Z[label="Z"];
U[label="Unobserved Confounders",observed="no"];
U->{X;Z};
X->{Y};
Z->{Y};
}"""

In [ ]:
rs = np.random.RandomState(42)
U = [] # социо-экономический статус
X = [] # лечение больного
Y = [] # успешное лечение
Z = [] # наличие лишнего веса

U+=[1]*100
U+=[0]*100

Z = [u if rs.uniform()<0.85 else rs.randint(low=0, high=2) for u in U] # 
X = [u if rs.uniform()<0.85 else rs.randint(low=0, high=2) for u in U]
Y = []
for x,z in zip(X,Z):
    # будем считать, что человек успешно вылечился, если назначено адекватное лечение и у него нет лишнего веса
    y = int((x + (1-z))/2) 
    Y.append(y)
    
    
dataframe = pd.DataFrame({'X':X, 'Y':Y, 'Z':Z})
dataframe.sample(5)

In [ ]:
model = CausalModel(
    data=dataframe,
    treatment=['X'],    
    outcome=["Y"])
model.interpret()

In [ ]:
print(model.identify_effect())

In [ ]:
print(
    model.estimate_effect(
        model.identify_effect(), 
        method_name='backdoor.linear_regression', 
        test_significance=True))

In [ ]:
model = CausalModel(
    data=dataframe,
    treatment=['X'],    
    outcome=["Y"], 
    graph=causal_graph)
model.interpret()

In [ ]:
print(model.identify_effect())

In [ ]:
print(
    model.estimate_effect(
        model.identify_effect(), 
        method_name='backdoor.linear_regression', 
        test_significance=True))

## И снова курение

In [ ]:
smoke = []
cancer = []
tar = []

smoke += [1] * 323
cancer += [0]*323
tar+=[1]*323

smoke+=[0]
cancer+=[0]
tar+=[1]

smoke+=[1]*57
cancer+=[1]*57
tar+=[1]*57

smoke+=[0]*19
cancer+=[1]*19
tar+=[1]*19


smoke+=[1]*18
cancer+=[0]*18
tar+=[0]*18

smoke+=[0]*38
cancer+=[0]*38
tar+=[0]*38

smoke+=[1]*2
cancer+=[1]*2
tar+=[0]*2

smoke+=[0]*342
cancer+=[1]*342
tar+=[0]*342

dataframe = pd.DataFrame({'smoke':smoke, 'tar':tar, 'cancer':cancer})
dataframe.sample(5)

In [ ]:
model = CausalModel(
    data=dataframe,
    treatment=['smoke'],
    outcome=["cancer"])
model.interpret()

In [ ]:
print(model.identify_effect(proceed_when_unidentifiable=True))

In [ ]:
print(
    model.estimate_effect(
        model.identify_effect(proceed_when_unidentifiable=True), 
        method_name="backdoor.linear_regression", ))


Получением, что пристрастие к курению сильно снижает вероятность рака)))

In [ ]:
causal_graph = """digraph {
smoke[label="smoke"];
tar[label="tar"];
cancer[label="cancer"];
U[label="Unobserved Confounders",observed="no"];
U->{smoke;cancer};
tar->{cancer};
smoke->{tar};
}"""

In [ ]:
model = CausalModel(
    data=dataframe,
    treatment=['smoke'],
    outcome=["cancer"],    
    instruments=["smoke"],
    graph=causal_graph)
model.interpret()

In [ ]:
print(model.identify_effect(proceed_when_unidentifiable=True))

In [ ]:
print(
    model.estimate_effect(
        model.identify_effect(proceed_when_unidentifiable=True), 
        method_name="frontdoor.two_stage_regression"))